# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# Step 1: Define stock tickers
high_risk_tickers = ['TSLA', 'NVDA', 'PLTR', 'COIN', 'ARKK']
low_risk_tickers = ['JNJ', 'PG', 'KO', 'WMT', 'MCD']
all_tickers = high_risk_tickers + low_risk_tickers

# Step 2: Download adjusted close prices
data = yf.download(all_tickers, start="2024-05-01", end="2025-05-01")

# If columns are multi-level, extract 'Adj Close' properly
if isinstance(data.columns, pd.MultiIndex):
    data = data['Adj Close']
else:
    raise ValueError("Unexpected column structure in downloaded data.")

# Step 3: Compute daily returns
returns = data.pct_change().dropna()
returns_high = returns[high_risk_tickers]
returns_low = returns[low_risk_tickers]

# Step 4: Equal weight portfolio metrics
def portfolio_metrics(returns, name="Portfolio"):
    n_assets = returns.shape[1]
    weights = np.array([1/n_assets] * n_assets)

    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252

    port_return = np.dot(weights, mean_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = port_return / port_vol

    cum_returns = (returns + 1).cumprod()
    max_dd = ((cum_returns.cummax() - cum_returns) / cum_returns.cummax()).max().max()

    return {
        "Portfolio": name,
        "Annual Return": port_return,
        "Annual Volatility": port_vol,
        "Sharpe Ratio": sharpe,
        "Max Drawdown": max_dd
    }

# Step 5: Calculate and summarize metrics
metrics_high = portfolio_metrics(returns_high, name="High Risk")
metrics_low = portfolio_metrics(returns_low, name="Low Risk")

summary_df = pd.DataFrame([metrics_high, metrics_low])
print("\n📊 Portfolio Performance Summary:")
print(summary_df)

# Step 6: Plot cumulative returns
plt.figure(figsize=(12, 6))
(returns_high + 1).cumprod().mean(axis=1).plot(label='High Risk Portfolio')
(returns_low + 1).cumprod().mean(axis=1).plot(label='Low Risk Portfolio')
plt.title('Cumulative Returns (Equal Weighted Portfolios)')
plt.xlabel('Date')
plt.ylabel('Growth of $1')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

[*********************100%***********************]  10 of 10 completed


KeyError: 'Adj Close'